#Training model 3


In [1]:
import numpy as np
import cv2
import sys
import random
import glob
import re
import time
import os
from tqdm.notebook import tqdm as log_progress
from matplotlib import pyplot as plt

from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model, load_model

###Complete this basing on where you are running the notebook:

In [2]:
on_colab = True
EPOCHS = 55
LEN_TRAINING_DATA = 75000
if on_colab:
    !git clone https://github.com/andreorto98/scopa.git
    path_to_deck = './scopa/scopa/new_deck/'
if not on_colab:
    path_to_deck = './new_deck/'

Cloning into 'scopa'...
remote: Enumerating objects: 389, done.
remote: Counting objects: 100% (389/389), done.
remote: Compressing objects: 100% (306/306), done.
remote: Total 389 (delta 123), reused 337 (delta 76), pack-reused 0
Receiving objects: 100% (389/389), 3.67 MiB | 29.84 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [3]:
def transform_img(img, angle = 0, scale = 1, tr = (0,0)):
  input_img_shape = img.shape
  img_center = (int(input_img_shape[0]/2), int(input_img_shape[1]/2))
  trasl_mat = np.zeros((2,3))
  trasl_mat[0,2]=tr[0]
  trasl_mat[1,2]=tr[1]
  trasl_mat[0,0] = trasl_mat[1][1] = 1
  img = cv2.warpAffine(img, trasl_mat, img.shape[0:2], flags=cv2.INTER_LINEAR)
  rot_mat = cv2.getRotationMatrix2D(img_center, angle, scale)
  img = cv2.warpAffine(img, rot_mat, img.shape[0:2], flags=cv2.INTER_LINEAR)
  return img.reshape(input_img_shape)

def generate_cards(n=1, layers=3, ang = True, traslate=False):
    cards = np.random.randint(1,41,n, np.uint8)
    if ang:
        angles= np.random.uniform(0,360,n)
    else:
        angles = np.full(n, 0.)
    scales= np.random.uniform(0.9,1.1,n)
    if layers == 1:
        print(path_to_deck+"*.jpg")
        inp_images = sorted([file for file in glob.glob(path_to_deck+"*.jpg")], key = string_to_n_card)
        inp_images = [cv2.imread(file, cv2.IMREAD_GRAYSCALE).reshape(256,256,1) for file in inp_images]
    elif layers == 3:
        inp_images = sorted([file for file in glob.glob("./deck0/*.jpg")], key = string_to_n_card)
        inp_images = [cv2.imread(file)[232:488,232:488] for file in inp_images]
    image_shape = inp_images[0].shape

    if traslate:
        marg = 5
        max_tr_x = int(image_shape[0]/2-image_shape[0]/marg)
        max_tr_y = int(image_shape[1]/2-image_shape[1]/marg)
        tr_x = np.random.randint(0,max_tr_x,n)
        tr_y = np.random.randint(0,max_tr_y,n)
    else:
        tr_x = np.zeros(n)
        tr_y = np.zeros(n)

    images = [transform_img(inp_images[cards[i]-1], angles[i], scales[i], (tr_x[i], tr_y[i]))
              for i in log_progress(range(len(cards)))]

    return np.array(images), np.array(cards)


def string_to_n_card(str):
    if str.startswith('.'):
         words = re.split('_|/|.j', str)[-3:-1]
    else: words = str.split('_')
    suit_to_n = {'hearts':0, 'diamonds':10, 'clubs':20, 'spades':30}
    return int(words[0])+suit_to_n[words[1]]

In [4]:
!ls

sample_data  scopa


In [5]:
n_samples = LEN_TRAINING_DATA

# con 10000 impiega 70s per laprima parte e usa circa 10 GB

print('start time1')
start = time.time()

# Generate training data1
train_data, target = generate_cards(n_samples, 1, True)

print(train_data.shape, target.shape)

print('cards_generated')
print("--- %s seconds ---" % (time.time() - start))

start time1
./scopa/scopa/new_deck/*.jpg



(75000, 256, 256, 1) (75000,)
cards_generated
--- 76.9101209640503 seconds ---


In [6]:
print(f'Dimension of train_data (MB): {sys.getsizeof(train_data)/1e6:.0f}', train_data.shape)

def card_to_n(card):
    target_n = [0.]*10
    target_n[(card-1)%10]=1.
    return target_n

def card_to_s(card):
    if card<11:
        return [1.,0.,0.,0.]
    elif card<21:
        return [0.,1.,0.,0.]
    elif card<31:
        return [0.,0.,1.,0.]
    else:
        return [0.,0.,0.,1.]
    

model1 = True
if model1:
    # creating the ground truth labels
    labels_n = np.array([card_to_n(card) for card in target])
    labels_s = np.array([card_to_s(card) for card in target])
    labels = [labels_n, labels_s]
    
  
    # building the model: suit
    inputs = Input(shape=train_data[0].shape)
    conv = Conv2D(3, (2,2), activation = 'relu', input_shape=train_data[0].shape)(inputs)
    pool = MaxPooling2D(pool_size=(2,2))(conv)
    conv = Conv2D(2, (2,2), activation = 'relu')(pool)
    pool = MaxPooling2D(pool_size=(2,2))(conv)
    conv = Conv2D(2, (2,2), activation = 'relu')(pool)
    pool = MaxPooling2D(pool_size=(3,3))(conv)

    flatten = Flatten()(pool)
    dense = Dense(64, activation='relu')(flatten)

    output1 = Dense(10,activation='softmax')(dense)
    output2 = Dense(4,activation='softmax')(dense)
    outputs= [output1,output2]
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss=["categorical_crossentropy", "categorical_crossentropy"], optimizer='adam',metrics=['categorical_accuracy'])

    model.summary()
    history = model.fit(train_data, labels , validation_split=0.8, epochs= EPOCHS, verbose=1)
    
    print('end of training')
    print("--- %s seconds ---" % (time.time() - start))


Dimension of train_data (MB): 4915 (75000, 256, 256, 1)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 255, 255, 3)  15          input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 127, 127, 3)  0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 126, 126, 2)  26          max_pooling2d[0][0]              
______________________________________

###check if val_accuracy is acceptable 

In [22]:
if history.history['val_dense_2_categorical_accuracy'][-1] < 0.96:
  history2 = model.fit(train_data, labels , validation_split=0.8, epochs= 5, verbose=1)

Epoch 1/5
469/469 [==============================] - 13s 28ms/step - loss: 0.0495 - dense_1_loss: 0.0079 - dense_2_loss: 0.0416 - dense_1_categorical_accuracy: 0.9978 - dense_2_categorical_accuracy: 0.9877 - val_loss: 0.0747 - val_dense_1_loss: 0.0032 - val_dense_2_loss: 0.0715 - val_dense_1_categorical_accuracy: 0.9991 - val_dense_2_categorical_accuracy: 0.9794
Epoch 2/5
469/469 [==============================] - 13s 28ms/step - loss: 0.0080 - dense_1_loss: 3.1024e-04 - dense_2_loss: 0.0076 - dense_1_categorical_accuracy: 0.9999 - dense_2_categorical_accuracy: 0.9979 - val_loss: 0.0692 - val_dense_1_loss: 7.1379e-04 - val_dense_2_loss: 0.0685 - val_dense_1_categorical_accuracy: 0.9997 - val_dense_2_categorical_accuracy: 0.9803
Epoch 3/5
469/469 [==============================] - 13s 28ms/step - loss: 0.0019 - dense_1_loss: 2.2956e-05 - dense_2_loss: 0.0019 - dense_1_categorical_accuracy: 1.0000 - dense_2_categorical_accuracy: 0.9997 - val_loss: 0.0577 - val_dense_1_loss: 6.9517e-04 - 

In [23]:
test_data, target = generate_cards(int(n_samples/10), 1, True)
test_pred0 = model.predict(test_data) 
      # this is a list whit 2 arrays of len(target) lenght: the first corresponds to number


pred_n = np.array([test_pred0[0][i] for i in range(len(target))])
pred_s = np.array([test_pred0[1][i] for i in range(len(target))])


test_pred_n = np.array([np.argmax(pred_n[i]) for i in range(len(target))])
test_pred_s = np.array([np.argmax(pred_s[i]) for i in range(len(target))])


test_label_n = np.array([card_to_n(card) for card in target])
test_label_n = np.array([np.argmax(test_label_n[i]) for i in range(len(target))])
test_label_s = np.array([card_to_s(card) for card in target])
test_label_s = np.array([np.argmax(test_label_s[i]) for i in range(len(target))])


from sklearn import metrics

print(f'Accuracy test set (n):\t{metrics.accuracy_score(test_label_n, test_pred_n):.1%}')
print(f'Accuracy test set (s):\t{metrics.accuracy_score(test_label_s, test_pred_s):.1%}')


print('----------------------------------------------')

./scopa/scopa/new_deck/*.jpg



Accuracy test set (n):	100.0%
Accuracy test set (s):	98.7%
----------------------------------------------


In [26]:
save = False
if save:
  if on_colab:
    model.save('./scopa/scopa/new_models/all_model_2.h5') 
  else:
    model.save('./new_models/all_model_2.h5')